# Series API Notebook

In [ ]:
import json, io, inspect, platform, sys
from pathlib import Path
import numpy as np
import pandas as pd

PANDAS_VERSION = pd.__version__
PYTHON_VERSION = sys.version
PLATFORM_INFO = platform.platform()
print('pandas:', PANDAS_VERSION)
print('python:', PYTHON_VERSION)
print('platform:', PLATFORM_INFO)
if not PANDAS_VERSION.startswith('2.2.'):
    print('WARNING: pandas version != 2.2.*; inventory may differ.')


In [ ]:
# BASE_DATASETS
rng = np.random.default_rng(42)
n_sales = 120
sales_dates = pd.date_range('2024-01-01', periods=90, freq='D')
df_sales = pd.DataFrame({
    'date': rng.choice(sales_dates, size=n_sales),
    'store_id': rng.integers(1, 11, size=n_sales),
    'customer_id': rng.integers(1000, 1101, size=n_sales),
    'sku': [f'SKU-{i:03d}' for i in rng.integers(1, 31, size=n_sales)],
    'qty': rng.integers(1, 6, size=n_sales),
    'price': rng.uniform(1.0, 100.0, size=n_sales).round(2),
    'discount': np.where(rng.random(size=n_sales) < 0.65, 0.0, rng.uniform(0.01, 0.30, size=n_sales)).round(2),
})
df_sales['date'] = pd.to_datetime(df_sales['date'])

df_customers = pd.DataFrame({
    'customer_id': np.arange(1000, 1120),
    'segment': pd.Categorical(rng.choice(['consumer', 'smb', 'enterprise'], size=120, replace=True)),
    'city': rng.choice(['Rome', 'Milan', 'Turin', 'Naples', 'Bologna'], size=120, replace=True),
    'signup_date': pd.to_datetime('2023-01-01') + pd.to_timedelta(rng.integers(0, 365, size=120), unit='D'),
})

df_events = pd.DataFrame({
    'ts': pd.to_datetime('2024-02-01') + pd.to_timedelta(rng.integers(0, 60*24*20, size=150), unit='m'),
    'user_id': rng.integers(1000, 1120, size=150),
    'event': pd.Categorical(rng.choice(['view', 'click', 'purchase', 'refund'], size=150, replace=True)),
    'duration_ms': rng.integers(0, 30001, size=150),
})

s_qty = df_sales['qty'].copy()
idx_dates = pd.Index(df_sales['date'].sort_values().unique(), name='date')

assert 50 <= len(df_sales) <= 200
assert str(df_sales['date'].dtype).startswith('datetime64')
assert np.issubdtype(df_sales['qty'].dtype, np.integer)
assert np.issubdtype(df_sales['price'].dtype, np.floating)
assert str(df_customers['signup_date'].dtype).startswith('datetime64')
assert str(df_events['ts'].dtype).startswith('datetime64')

df_sales.head()


In [ ]:
target_obj = s_qty.copy()

In [ ]:
covered_members = set()
skipped_members = {}
missing_from_previous_run = []
if missing_from_previous_run:
    print('Missing from previous run:', len(missing_from_previous_run))


## pandas.Series.astype

- **Categoria**: Cleaning
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `astype` all'oggetto `Series` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'astype'
qualname = 'pandas.Series.astype'
try:
    member = getattr(target_obj, member_name)
    if 'callable' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'callable' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])



## pandas.Series.at

- **Categoria**: Indexing
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `at` all'oggetto `Series` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'at'
qualname = 'pandas.Series.at'
try:
    member = getattr(target_obj, member_name)
    if 'indexer_accessor' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'indexer_accessor' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])



## pandas.Series.cat

- **Categoria**: Other
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `cat` all'oggetto `Series` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'cat'
qualname = 'pandas.Series.cat'
try:
    member = getattr(target_obj, member_name)
    if 'indexer_accessor' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'indexer_accessor' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])



## pandas.Series.dt

- **Categoria**: Other
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `dt` all'oggetto `Series` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'dt'
qualname = 'pandas.Series.dt'
try:
    member = getattr(target_obj, member_name)
    if 'indexer_accessor' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'indexer_accessor' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])



## pandas.Series.fillna

- **Categoria**: Cleaning
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `fillna` all'oggetto `Series` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'fillna'
qualname = 'pandas.Series.fillna'
try:
    member = getattr(target_obj, member_name)
    if 'callable' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'callable' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])



## pandas.Series.groupby

- **Categoria**: Aggregation
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `groupby` all'oggetto `Series` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'groupby'
qualname = 'pandas.Series.groupby'
try:
    member = getattr(target_obj, member_name)
    if 'callable' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'callable' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])



## pandas.Series.head

- **Categoria**: Other
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `head` all'oggetto `Series` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'head'
qualname = 'pandas.Series.head'
try:
    member = getattr(target_obj, member_name)
    if 'callable' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'callable' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])



## pandas.Series.iat

- **Categoria**: Indexing
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `iat` all'oggetto `Series` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'iat'
qualname = 'pandas.Series.iat'
try:
    member = getattr(target_obj, member_name)
    if 'indexer_accessor' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'indexer_accessor' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])



## pandas.Series.iloc

- **Categoria**: Indexing
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `iloc` all'oggetto `Series` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'iloc'
qualname = 'pandas.Series.iloc'
try:
    member = getattr(target_obj, member_name)
    if 'indexer_accessor' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'indexer_accessor' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])



## pandas.Series.loc

- **Categoria**: Indexing
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `loc` all'oggetto `Series` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'loc'
qualname = 'pandas.Series.loc'
try:
    member = getattr(target_obj, member_name)
    if 'indexer_accessor' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'indexer_accessor' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])



## pandas.Series.rolling

- **Categoria**: Window
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `rolling` all'oggetto `Series` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'rolling'
qualname = 'pandas.Series.rolling'
try:
    member = getattr(target_obj, member_name)
    if 'callable' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'callable' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])



## pandas.Series.str

- **Categoria**: Other
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `str` all'oggetto `Series` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'str'
qualname = 'pandas.Series.str'
try:
    member = getattr(target_obj, member_name)
    if 'indexer_accessor' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'indexer_accessor' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])



## pandas.Series.tail

- **Categoria**: Other
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `tail` all'oggetto `Series` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'tail'
qualname = 'pandas.Series.tail'
try:
    member = getattr(target_obj, member_name)
    if 'callable' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'callable' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])



## pandas.Series.value_counts

- **Categoria**: Aggregation
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `value_counts` all'oggetto `Series` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'value_counts'
qualname = 'pandas.Series.value_counts'
try:
    member = getattr(target_obj, member_name)
    if 'callable' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'callable' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])



In [ ]:
payload = {
    'notebook': '02_Series.ipynb',
    'covered_members': sorted(covered_members),
    'skipped_members': skipped_members,
}
Path('coverage_series.json').write_text(json.dumps(payload, indent=2), encoding='utf-8')
print('Wrote coverage_series.json with', len(covered_members), 'covered and', len(skipped_members), 'skipped')
